## Guide for the use of `sns_modeling` package

This notebook provides an overview of how to install and use the package to build and solve separation network synthesis problems 

### 1. Setup and install

Download or clone the package from the Github repo [sns_modeling](https://github.com/pfauk/sns_modeling). In a terminal, navigate to the location directory location and install the package dependencies by running: 

```
pip install -r requirements.txt
```

**Important**: this package requires an installed version of Gurobi to solve the MIQCP model. It is possible to pip install Gurobi from the Python Package Index (PyPI). However, the free version comes with a trial license that will only be able to solve models of a smaller size (2,000 variables or constraints).

After installing dependencies, navigate to the directory of the package and run:

```
pip install -e . 
```

### 2. Package Structure / Overview

Most of the core functionality of the separation network model is contained in `src\thermal_coupled\therm_dist.py` and implemented in the `build_model` function that returns a Pyomo model. The `src\superstructure` directory contains the functionality to build the state task network (stn) for the superstructure of the problem. All problem data for a specific instance should be located in the `src\data` directory

### 3. Data Entry 

In order to build the mathematical program, the user must provide parameters for the overall system and the species present in the inlet feed. The data is parsed from spreadsheets under the `src\data` directory. Several examples are provided and it is recommended to just copy the existing sheets and edit values to create a new problem.

**System parameters**

The inlet molar flow rate $F_0$

### 4. Building a Problem

Here we will show an example of building and solving a problem with a system feed with 4 components and splits between consecutive key components. Not all of the below import statements are required. Many are used to save and inspect the model after completion.

In [1]:
# import statements 

import logging
import pyomo.environ as pyo
from pyomo.util.infeasible import log_infeasible_constraints, find_infeasible_constraints
from pyomo.util.model_size import build_model_size_report
from utils import (
    Data,
    get_model_type,
    pprint_network,
    pprint_tasks,
    save_model_to_file,
    save_solution_to_file,
    get_model_type,
    print_constraint_type)
from superstructure.stn import stn
from superstructure.stn_nonconsecutive import stn_nonconsecutive
from thermal_coupled.therm_dist import build_model

First, you will import data from the data file (excel) located in the `src\data\` directory. The `Data` class from the `utils.py` script will be used to hold all the problem data and pass it as an arguement to the `build_model` function.

In [17]:
# specify number of components and data file name
n = 4
data_file_name = '4_comp_hydrocarbon_1.xlsx'

# import problem data for system and relevant species to data object
mixture_data = Data(data_file_name)

# can inspect the Data object
print()
print('Inlet data')
print('================================================================')
print(mixture_data.system_df)

print()
print('Mixture species data')
print('================================================================')
print(mixture_data.species_df)

Next, build the problem superstucture by specifying the number of components. The `superstructure\stn.py` script contains functionality for state task network with splits between both consecutive and non-consecutive components. The class `stn` builds a tree and index sets for splits between consecutive key components. You could also choose to use `stn_nonconsecutive` imported from `superstructure\stn_nonconsecutive.py`

In [3]:
# build state-task network superstrucutre and associated index sets
network_superstructure = stn(n)
network_superstructure.generate_tree()
network_superstructure.generate_index_sets()

![stn consecutive splits](./images/consecutive_split_stn.png)

*A state task network for a 4 component mixture with splits between **consecutive** key components*

![stn nonconsecutive splits](./images/nonconsecutive_split_stn.png)

*A state task network for a 4 component mixture with splits between **nonconsecutive** key components*

In [4]:
# can visualize the superstructure if desired
network_superstructure.print_tree()

State(ABCD)
  Task(A/BCD)
    State(A, final=True)
    State(BCD)
      Task(B/CD)
        State(B, final=True)
        State(CD)
          Task(C/D)
            State(C, final=True)
            State(D, final=True)
      Task(BC/D)
        State(BC)
          Task(B/C)
            State(B, final=True)
            State(C, final=True)
        State(D, final=True)
  Task(AB/CD)
    State(AB)
      Task(A/B)
        State(A, final=True)
        State(B, final=True)
    State(CD)
      Task(C/D)
        State(C, final=True)
        State(D, final=True)
  Task(ABC/D)
    State(ABC)
      Task(A/BC)
        State(A, final=True)
        State(BC)
          Task(B/C)
            State(B, final=True)
            State(C, final=True)
      Task(AB/C)
        State(AB)
          Task(A/B)
            State(A, final=True)
            State(B, final=True)
        State(C, final=True)
    State(D, final=True)


Now build the Pyomo model.

In [5]:
model = build_model(network_superstructure, mixture_data)

# returns a Pyomo Concrete Model object
print(type(model))

<class 'pyomo.core.base.PyomoModel.ConcreteModel'>


It can sometimes be useful to inspect the model object throughout the workflow. The model that is constructed by the `build_model` function is a generalized disjunctive program (GDP). The `get_model_type` from `utils.py` allows you to see what type of mathematical model the Pyomo model object contains. Furthermore, the `save_model_to_file` function can be used to create a text file to inspect the entire model object in a pretty printed format. Saving the model for inspection is best done prior to transforming of the GDP model. By default, the pretty printed Pyomo model is saved to `thermal_coupled\saved_models`.

In [6]:
# saving the pyomo model to a file
save_model_to_file(model, '4_comp_model')

In [14]:
# check the model type prior to transformation
print(f'Model type before transformation: {get_model_type(model)}')

# use of Pyomo.GDP to apply Big-M transformation
pyo.TransformationFactory('core.logical_to_linear').apply_to(model)

mbigm = pyo.TransformationFactory('gdp.bigm')
mbigm.apply_to(model)

print(f'Model type after transformation: {get_model_type(model)}')

### 4. Solution and Output

After applying the Big-M transformation using Pyomo.GDP's TransformationFactory, the Pyomo model is a non-convex mixed-integer quadratically constrained program (MIQCP). We can use Gurobi to solve the model. Gurobi has a number of parameters that can be passed to the solver. A [full list of solver parameters](https://www.gurobi.com/documentation/current/refman/parameters.html) can be founds on the Gurobi website. For now we recommend setting the NumericFocus and nonConvex parameters to values of 2.

In [15]:
# Pyomo solver factory
solver = pyo.SolverFactory('gurobi')

# Gurobi solver options
solver.options = {'NumericFocus': 2,
                  'nonConvex': 2}

Now send the Pyomo model to the solver. The logging setup helps to troubleshoot any infeasible constraints that might exist in the model. It is not uncommon to have some infeasible log statements as a result of some of the transformation variables.

In [16]:
results = solver.solve(model, tee=True)

# Log infeasible constraints if any
logging.basicConfig(level=logging.INFO)
log_infeasible_constraints(model)
find_infeasible_constraints(model)

INFO:pyomo.util.infeasible:CONSTR _pyomo_gdp_bigm_reformulation.relaxedDisjuncts[0].transformedConstraints['intermediate_var_con[C,r1]_32',(C,r1),lb]: 1.0 </= 0.9999986519279662
INFO:pyomo.util.infeasible:CONSTR _pyomo_gdp_bigm_reformulation.relaxedDisjuncts[0].transformedConstraints['intermediate_var_con[D,r1]_34',(D,r1),lb]: 1.0 </= 0.9999987956263393
INFO:pyomo.util.infeasible:CONSTR _pyomo_gdp_bigm_reformulation.relaxedDisjuncts[6].transformedConstraints['intermediate_var_con[D,r1]_40',(D,r1),lb]: 1.0 </= 0.9999983298108829
INFO:pyomo.util.infeasible:CONSTR _pyomo_gdp_bigm_reformulation.relaxedDisjuncts[10].transformedConstraints['intermediate_var_con[C,r1]_40',(C,r1),lb]: 1.0 </= 0.9999985998324995
INFO:pyomo.util.infeasible:CONSTR _pyomo_gdp_bigm_reformulation.relaxedDisjuncts[10].transformedConstraints['intermediate_var_con[C,r3]_44',(C,r3),lb]: 1.0 </= 0.9999985248596212


<generator object find_infeasible_constraints at 0x000001D7DEF3AA40>

To see the output of the solution, just use the `print_network` function from the package utilities

In [20]:
pprint_network(model)

You can save the pretty printed solution output to a text file with the used of `save_solution_to_file`. The `thermal_coupled\results` directory is the default save location.

In [ ]:
save_solution_to_file(model, '4_comp_solution_1')